In [214]:
import pixiedust

In [215]:
import pandas as pd

In [216]:
df = pd.read_csv('operations-report-hdrvg(1).csv')

In [217]:
display(df)

Items,Origin,Items grouped,Item group filter,Category,Unit,amount,Mission,District,VDC,Village/Ward/extra info,VDC Latitude,VDC Longitude,District Latitude,District Longitude,Date,Location name
Sanitary pads,nan,Sanitary pads,nan,Sanitation - supplies,box,2.0,MISSION 07/5 *05,Nuwakot,Sikre,nan,27.83895,85.37597,27.964000000000002,83.0549,7-May-15,"Sikre,"
Diclofenac gel,nan,Medicine - drug,24.0,Medicine,unit,2.0,MISSION 30/4 nosheet 14,Sindhupalchok,Sangachok,Jogitar,27.695320000000002,85.72073,27.829359999999998,85.54504,30-Apr-15,"Sangachok, Jogitar"
Jeevanjal,nan,Medicine - rehydration,3.0,Medicine,unit,50.0,MISSION 30/4 nosheet 11,Lalitpur,Bungamati,nan,27.6211,85.303,27.58333,85.33333,30-Apr-15,"Bungamati,"
Noodles,nan,Noodles,2.0,Food - Dry Food,box,4.0,MISSION 01/5 nosheet 07,Lalitpur,Devichour,"and also Dhusel, Chhampi, Dukuchhap",27.5635,85.28299999999999,27.58333,85.33333,1-May-15,"Devichour, and also Dhusel, Chhampi, Dukuchhap"
Medical Kit,nan,Medical Kit,nan,Medicine,unit,2.0,MISSION 07/5 *05,Nuwakot,Sikre,nan,27.83895,85.37597,27.964000000000002,83.0549,7-May-15,"Sikre,"
Solar Lamps/Chargers (UNHCR),UNHCR,Solar Lamps/Chargers (UNHCR),nan,Shelter,unit,18.0,MISSION 16/5 *02,Khotang,RakhaBangdel,"R.Maheswori, RakhaDipsung, Sungdel",27.384,86.8205,27.25,86.83333,16-May-15,"RakhaBangdel, R.Maheswori, RakhaDipsung, Sungdel"
Rice,nan,Rice,nan,Food - Dry Food,sack,20.0,MISSION 15/5 *07,Ramechhap,Chuchure,Shivalaya village,27.61682,86.33256999999999,27.3303,86.06458,15-May-15,"Chuchure, Shivalaya village"
Sanitary pads,nan,Sanitary pads,nan,Sanitation - supplies,pack,460.0,MISSION 06/5 *02,Sindhupalchok,Mahankal,Sera,27.886940000000003,85.51056,27.829359999999998,85.54504,6-May-15,"Mahankal, Sera"
Sanitary pads,nan,Sanitary pads,nan,Sanitation - supplies,box,1.0,MISSION 22/5 *05,Nuwakot,Mahakali,Nepane ward 9,27.87254,85.36865999999999,27.964000000000002,83.0549,22-May-15,"Mahakali, Nepane ward 9"
paraflam,nan,Medicine - drug,53.0,Medicine,unit,40.0,MISSION 01/5 nosheet 04,Lalitpur,Bhardev,nan,27.5493,85.3846,27.58333,85.33333,1-May-15,"Bhardev,"


In [218]:
df = df.dropna(subset = ['VDC'], how='all')

In [219]:
df['Location name'] = df['VDC'] + ', ' + df['Village/Ward/extra info'].fillna('')
# df['Location name'].apply(lambda x: x[:-2] if x[-2:] == ', ' else x )[29]

In [220]:
df

,Items,Origin,Items grouped,Item group filter,Category,Unit,amount,"FSC - KGs (sack=25, pack=1, box=4.5)",FSC - Total KGs,Mission,District,VDC,Village/Ward/extra info,VDC Latitude,VDC Longitude,District Latitude,District Longitude,Date,Location name
0,NaN,NaN,Noodles,NaN,Food - Dry Food,box,5.0,4.5,22.5,MISSION 29/4 *01,Bhaktapur,Nankhel,Maligaun,27.64220,85.45980,27.66734,85.41673,29-Apr-15,"Nankhel, Maligaun"
1,NaN,NaN,Medicine - rehydration,NaN,Medicine,unit,1.0,Undefined,#VALUE!,MISSION 29/4 *01,Bhaktapur,Nankhel,Maligaun,27.64220,85.45980,27.66734,85.41673,29-Apr-15,"Nankhel, Maligaun"
2,NaN,NaN,Piyush,NaN,Sanitation - Water purification,bottle,1.0,Undefined,#VALUE!,MISSION 29/4 *01,Bhaktapur,Nankhel,Maligaun,27.64220,85.45980,27.66734,85.41673,29-Apr-15,"Nankhel, Maligaun"
3,NaN,NaN,Tarp,NaN,Shelter,unit,8.0,Undefined,#VALUE!,MISSION 29/4 *01,Bhaktapur,Nankhel,Maligaun,27.64220,85.45980,27.66734,85.41673,29-Apr-15,"Nankhel, Maligaun"
4,NaN,NaN,Tarp,NaN,Shelter,unit,5.0,Undefined,#VALUE!,MISSION 29/4 *02,Kabhre Palanchok,BanepaN.P.,Aashapuri,27.63300,85.53020,27.58333,85.66667,29-Apr-15,"BanepaN.P., Aashapuri"
5,NaN,NaN,Tarp,NaN,Shelter,unit,4.0,Undefined,#VALUE!,MISSION 29/4 *03,Bhaktapur,Sipadol,Soche,27.64714,85.43797,27.66734,85.41673,29-Apr-15,"Sipadol, Soche"
6,NaN,NaN,Medicine - rehydration,NaN,Medicine,unit,20.0,Undefined,#VALUE!,MISSION 29/4 *04,Bhaktapur,Bhaktapur,Durlang,27.66734,85.41673,27.66734,85.41673,29-Apr-15,"Bhaktapur, Durlang"
7,NaN,NaN,Tarp,NaN,Shelter,unit,20.0,Undefined,#VALUE!,MISSION 29/4 *04,Bhaktapur,BhaktapurN.P.,Durlang,27.67480,85.43190,27.66734,85.41673,29-Apr-15,"BhaktapurN.P., Durlang"
8,NaN,NaN,Medical Kit,NaN,Medicine,unit,1.0,Undefined,#VALUE!,MISSION 29/4 *05,Bhaktapur,Sipadol,Naichal,27.64714,85.43797,27.66734,85.41673,29-Apr-15,"Sipadol, Naichal"
9,NaN,NaN,Tarp,NaN,Shelter,unit,7.0,Undefined,#VALUE!,MISSION 29/4 *05,Bhaktapur,Sipadol,Naichal,27.64714,85.43797,27.66734,85.41673,29-Apr-15,"Sipadol, Naichal"


In [221]:
df = df.drop(['FSC - KGs (sack=25, pack=1, box=4.5)', 'FSC - Total KGs'], axis =1)

In [222]:
df['Items'] = df['Items'].fillna(df['Items grouped'])

In [223]:
df.to_csv('DB_ready.csv')

In [224]:
import requests
import csv
import json

In [225]:
# configuration values
config = dict(
    username="2660e8fa-aac2-45ef-81ba-217967728ad2-bluemix", 
    password="9cd1b00e6b6927e663bfa0e2e9d677e163378a7c98e09bfb598dd74a4439fc7a",
    database="cfc_patient_db")

# dict of table names and their csv representations
csv_lookup = {
    # "table_name": "path/to/file.csv"
    "Nepal": "DB_ready.csv"
}

# dict of request data, which we'll upload to Cloudant
requests_data = {}

for table, filepath in csv_lookup.items():
    request_data = dict(docs=[]) 
    # get our data
    with open(filepath, 'rt') as f:
        reader = csv.DictReader(f)
        # put into request body    
        for row in reader:
            row['type'] = table # add doctype based on table
            request_data['docs'].append(row)
    requests_data[table] = request_data


In [226]:
url = "https://{username}:{password}@{username}.cloudant.com/{database}/_bulk_docs".format(**config)

In [227]:
r = requests.post(url)
print(r.text)

{"error":"bad_content_type","reason":"Content-Type must be application/json"}



In [228]:
auth_headers = {"Content-Type": "application/x-www-form-urlencoded"}
r = requests.post(url= url, json=request_data)

In [229]:
r.text

'[{"ok":true,"id":"5354af5c52e35b07bd695d11ec72903b","rev":"1-d2d566152570b7ac8e667c42b274f9c6"},{"ok":true,"id":"5354af5c52e35b07bd695d11ec729299","rev":"1-a831257d1ce43066708073378f35e3e9"},{"ok":true,"id":"5354af5c52e35b07bd695d11ec7297f7","rev":"1-3680ce6a67107b741c251270102fe9a4"},{"ok":true,"id":"5354af5c52e35b07bd695d11ec7298be","rev":"1-6dbfd65e42b3bcdb34db7d0e8e8d6702"},{"ok":true,"id":"5354af5c52e35b07bd695d11ec72a0e8","rev":"1-57c50bb5e0787a0c9d5c714f19cd3999"},{"ok":true,"id":"5354af5c52e35b07bd695d11ec72af3f","rev":"1-833bbecc4711e1f336974db36e51079d"},{"ok":true,"id":"5354af5c52e35b07bd695d11ec72af4f","rev":"1-e1a8ffd00797310677ba82de94a9dd21"},{"ok":true,"id":"5354af5c52e35b07bd695d11ec72b63f","rev":"1-c3e9c59eb03df1ddcf67b2d7cfb9fdc8"},{"ok":true,"id":"5354af5c52e35b07bd695d11ec72b93d","rev":"1-728a23d4df7eac3f49e7442e5d9e9cd1"},{"ok":true,"id":"5354af5c52e35b07bd695d11ec72bf52","rev":"1-d3bc7f3836e95d4e644af4088749bef7"},{"ok":true,"id":"5354af5c52e35b07bd695d11ec72c68